In [ ]:
!pip install --upgrade --quiet langchain-community langchain-openai neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["NEO4J_URI"] = ""
os.environ["NEO4J_USERNAME"] = ""
os.environ["NEO4J_PASSWORD"] = ""

In [ ]:
!pip install pandas

In [ ]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

# greate graph with the information

movies_query = """
CREATE
(red:Colour {name: 'red'}),
(green:Colour {name: 'green'}),
(blue:Colour {name: 'blue'}),
(white:Colour {name: 'white'}),
(black:Colour {name: 'black'}),
(yellow:Colour {name: 'yellow'}),
(orange:Colour {name: "orange"}),
(red)-[:grandparent]->(yellow),
(green)-[:siblings]->(yellow),
(green)-[:unknown]->(orange),
(black)-[:same_as]->(white),
(white)-[:parent]->(blue),
(white)-[:grandparent]->(orange),
(blue)-[:sibling]->(black);
"""

graph.query(movies_query)

In [ ]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Colour {name: STRING}
Relationship properties:

The relationships:
(:Colour)-[:sibling]->(:Colour)
(:Colour)-[:grandparent]->(:Colour)
(:Colour)-[:parent]->(:Colour)
(:Colour)-[:same_as]->(:Colour)
(:Colour)-[:siblings]->(:Colour)
(:Colour)-[:unknown]->(:Colour)


In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({"query": "what is the label of the relationship between black and white. dont forget to return"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c1:Colour {name: "black"})-[r]->(c2:Colour {name: "white"})
RETURN TYPE(r) AS relationship_type

Full Context:
[{'relationship_type': 'same_as'}]

> Finished chain.


In [ ]:
response['result']

'The label of the relationship between black and white is "same_as".'